<a href="https://colab.research.google.com/gist/ruvnet/81e00e2279c6fc0d604d4b2d70eb2482/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧬 BioForge: AI-Powered Synthetic Biology

Created by **rUv**, cause he could.

Welcome to **BioForge**, a hands-on guide to the **creation of synthetic life**. Advances in AI and biology are converging to enable the design of entire genomes, the creation of novel organisms, and even the rewriting of life itself. This is not just about Evo 2—it’s about harnessing technology to engineer biology from scratch.

We now have the capability to generate DNA from nothing, edit genomes with precision, and design life forms for medicine, sustainability, and exploration. The power to *write biology* is shifting from the slow process of evolution to rapid, intentional engineering.

---

## 🔬 What Can We Do?
- **Create synthetic genomes** that serve as blueprints for new life forms.
- **Predict mutations & genetic functions** to understand DNA at scale.
- **Design CRISPR edits** for precise control over genetic material.
- **Engineer new organisms**: from bacteria to plants, and even future space life.

---

## 🚀 Why It Matters
- **Accelerate Genetic Research:** AI speeds up discovery in genetics.
- **Medical Breakthroughs:** Rewrite DNA to treat and cure diseases.
- **Sustainable Biotechnology:** Engineer microbes for fuel, food, and environmental applications.
- **Terraforming Life:** Pave the way for designing life for other planets.

---

## ⚡ What You’ll Learn
1. **Installation:** Set up the necessary tools in Google Colab.
2. **DNA Generation:** Generate synthetic sequences using Evo 2.
3. **Editing & Engineering:** Modify genomes and simulate CRISPR edits.
4. **Evaluation & Ethics:** Assess DNA quality and consider responsible use.

---

## 🏥 Who Can Use This?
- **Biologists**: For gene design and studying synthetic life.
- **Medical Researchers**: To develop genomic therapies.
- **AI & ML Engineers**: Building models for synthetic biology.
- **Biotech & Space Research:** Engineering life for new frontiers.

Let's build the future of biology together. 🚀🧬

## 1️⃣ Installation: Setting Up the Environment

Before diving into genome generation, we need to install the necessary dependencies. This includes cloning the Evo 2 repository and installing its dependencies. The repository uses submodules, so we clone with the `--recurse-submodules` flag.

**Troubleshooting Tips:**
- If you encounter Git errors, ensure Colab's Git version is up-to-date.
- If pip installation fails, check that you're using a supported version of Python (ideally Python 3.11+).

In [ ]:
# Install system dependencies
!apt-get update
!apt-get install -y build-essential libssl-dev libffi-dev python3-dev

In [ ]:
# Upgrade pip and setuptools
!pip install --upgrade pip setuptools wheel

In [ ]:
# Evo 2 installation
!pip install -vvv ./evo2

### Verify Installation

Now, let's check that the environment is correctly set up by verifying the Python version and CUDA availability.

In [ ]:
import sys
import torch

print("Python version:", sys.version)
print("CUDA available:", torch.cuda.is_available())
print("GPU device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None")

## 2️⃣ Load Evo 2 Model & Generate Synthetic DNA

In this section, we load the Evo 2 model (using the 7B parameter version, which is well-suited for Colab) and generate a synthetic DNA sequence. Although Evo 2 is our reference model, our goal is to illustrate how AI can help in designing synthetic biology projects.

**Key Points:**
- The 7B model offers a balance between performance and resource usage.
- The model tokenizes a DNA prompt and generates additional nucleotides.
- GPU acceleration is crucial for running these large models efficiently.

In [ ]:
import torch
from evo2 import Evo2

# Load the Evo 2 7B model (ensure you have a GPU runtime)
evo2_model = Evo2("evo2_7b")
print("Model loaded successfully.")

### Generate a Synthetic DNA Sequence

Use a simple DNA prompt to generate a new sequence. You can modify the prompt, token count, and other parameters (like temperature and top_k) to experiment with different outputs.

In [ ]:
prompt = "ATGCGA"
num_tokens = 200  # Adjust for desired length

# Generate the sequence
generated_sequence = evo2_model.generate(prompt_seqs=[prompt], n_tokens=num_tokens).sequences[0]
print("Generated DNA Sequence:")
print(generated_sequence)

## 3️⃣ Evaluate the Generated Sequence

Biological evaluation is essential. In this section, we'll:
- **Calculate GC content:** Important for understanding genome stability.
- **Count start codon occurrences:** 'ATG' is a common start codon in many organisms.
- **Perform a simple sequence alignment:** Compare the generated sequence with a dummy reference to get a sense of similarity.

These metrics help ensure the synthetic DNA has realistic properties.

In [ ]:
from Bio.Seq import Seq
from Bio.SeqUtils import GC
from Bio import pairwise2

# Calculate GC content
gc_content = GC(generated_sequence)
print(f"GC content: {gc_content:.2f}%")

# Count occurrences of the start codon 'ATG'
start_codon = "ATG"
atg_count = generated_sequence.count(start_codon)
print(f"Occurrences of 'ATG': {atg_count}")

# Create a dummy reference (prompt + filler A's) and align
ref_seq = prompt + "A" * (len(generated_sequence) - len(prompt))
alignment = pairwise2.align.globalxx(generated_sequence, ref_seq)
print(f"Alignment Score: {alignment[0].score}")

## 4️⃣ Natural Language UI & Function Calling

In this section, we add a natural language interface that lets you describe what you want to create. We'll use the OpenRouter API with the **o3-mini-high** model and leverage function calling to map your instruction to specific functions (for example, generating a CRISPR guide or computing GC content).

### Key Points:
- **Google Colab Secrets:** Your OpenRouter API key is securely stored as a secret named `OPENROUTER_API_KEY`.
- **Function Calling:** The system uses natural language to decide which function to invoke.
- **GPU Requirement:** Remember, a GPU runtime is required for efficient processing.

Below is the complete implementation.

In [ ]:
import os
import json
import requests
import ipywidgets as widgets
from IPython.display import display, clear_output

# Ensure GPU is available
import torch
assert torch.cuda.is_available(), "This notebook requires a GPU-enabled runtime."

# Retrieve the OpenRouter API key from Colab secrets
api_key = os.environ.get("OPENROUTER_API_KEY")
if not api_key:
    raise Exception("API key not found. Please add your 'OPENROUTER_API_KEY' in Colab secrets.")

# Define the OpenRouter API endpoint for the o3-mini-high model
api_url = "https://api.openrouter.ai/v1/engines/o3-mini-high/completions"

# Set up the request headers
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}

# Define functions that can be invoked via natural language instructions

def generate_crispr_guide(sequence, window=50):
    """
    Dummy function to simulate CRISPR guide RNA design.
    Returns a mock guide RNA based on the input sequence.
    """
    guide = sequence[:20]
    cut_position = len(guide)
    return f"Guide RNA: {guide}, Cut at position: {cut_position}"


def compute_gc_content(sequence):
    """
    Computes and returns the GC content of the given DNA sequence.
    """
    gc = sum(1 for base in sequence if base in 'GCgc')
    return f"GC Content: {gc / len(sequence) * 100:.2f}%"

# Function router to map natural language instructions to our functions

def function_router(instruction, sequence):
    instruction = instruction.lower()
    if "crispr" in instruction or "guide" in instruction:
        return generate_crispr_guide(sequence)
    elif "gc content" in instruction or "gc%" in instruction:
        return compute_gc_content(sequence)
    else:
        return "No function matched the instruction."

# Create UI elements with ipywidgets
prompt_box = widgets.Textarea(
    value='Describe what you want to create (e.g., "Generate a CRISPR guide for this sequence" or "What is the GC content?")',
    placeholder='Type your design instruction here',
    description='Instruction:',
    layout=widgets.Layout(width='100%', height='100px')
)

# Button to trigger the instruction
submit_button = widgets.Button(description="Submit Instruction", button_style='success')

# Output widget to display API and function results
output_area = widgets.Output()


def on_submit(button):
    with output_area:
        clear_output()
        instruction = prompt_box.value
        print("Instruction:", instruction)

        # Prepare payload for OpenRouter API call with function calling enabled (hypothetical flag)
        payload = {
            "prompt": instruction,
            "max_tokens": 100,
            "temperature": 1.0,
            "top_k": 10,
            "function_call": true
        }

        # Make API call to OpenRouter
        response = requests.post(api_url, headers=headers, data=json.dumps(payload))
        if response.status_code == 200:
            data = response.json()
            completion = data.get("completion", "No completion returned.")
            print("API Completion:", completion)

            # Simulate function calling by routing the instruction locally
            result = function_router(instruction, generated_sequence)
            print("Function Calling Result:", result)
        else:
            print("Error:", response.status_code, response.text)

submit_button.on_click(on_submit)

display(prompt_box, submit_button, output_area)

## 5️⃣ Ethical Considerations & Best Practices

When engineering synthetic life, it's essential to consider ethical implications and safety protocols:

- **Responsible Use:** Ensure that any synthetic biology project adheres to strict biosecurity and ethical guidelines.
- **Screening:** Always screen generated sequences for potential pathogenic elements before any practical application.
- **Transparency:** Document your work and share findings responsibly.

Refer to guidelines from institutions like the NIH and relevant ethical boards for further reading.

## 🎯 Conclusion

In this tutorial, we have:
- Installed and set up the Evo 2 environment in Google Colab.
- Loaded a pre-trained Evo 2 model and generated synthetic DNA sequences.
- Evaluated the sequences with key biological metrics.
- Integrated a natural language UI using the OpenRouter API and function calling for interactive design.
- Discussed ethical considerations and best practices for synthetic biology.

The future of engineering life is here. Experiment, innovate, and build responsibly. **Created by rUv, because he could.** 🚀